In [31]:
import torch
import paddle
import collections
import typing
import json
import itertools
import operator

In [2]:
print(torch.__version__)
print(paddle.__version__)

1.7.0+cu101
2.0.0-rc1


# Device

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [4]:
print(paddle.device.get_cudnn_version())
print(paddle.device.get_device())

data = torch.rand(2, 2)
print(data.device)

7604
gpu:0
cpu


In [7]:
device = paddle.set_device('cpu')
print(device)
print(type(device))

data = paddle.rand(shape=(2, 2))
print(data)

CPUPlace
<class 'paddle.fluid.core_avx.CPUPlace'>
Tensor(shape=[2, 2], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[0.72883028, 0.14966780],
        [0.56501168, 0.48528844]])


# Representation

In [8]:
# nn.Module.__repr__
# def extra_repr(self) -> str:

torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2, padding=1, )

Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

In [9]:
# None
paddle.nn.Conv2D(3, 32, kernel_size=(3, 2), stride=2, padding=1)

In [10]:
class MM(torch.nn.Module):
    '''
    '''
    def __init__(self, ):
        super(MM, self).__init__()
        
        self.f1 = torch.nn.Linear(10, 10)
        self.conv1 = torch.nn.Conv2d(3, 10, 3, 2)
        self.seq1 = torch.nn.Sequential(torch.nn.Linear(10, 10), torch.nn.Conv2d(3, 10, 3, 2)) 
        self.seq2 = torch.nn.Sequential(self.seq1, torch.nn.Conv2d(3, 10, 3, 2))
    def forward(self, ):
        '''
        '''
        pass
MM()

MM(
  (f1): Linear(in_features=10, out_features=10, bias=True)
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(2, 2))
  (seq1): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Conv2d(3, 10, kernel_size=(3, 3), stride=(2, 2))
  )
  (seq2): Sequential(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): Conv2d(3, 10, kernel_size=(3, 3), stride=(2, 2))
    )
    (1): Conv2d(3, 10, kernel_size=(3, 3), stride=(2, 2))
  )
)

In [11]:
class MM(paddle.nn.Layer):
    '''
    '''
    def __init__(self, ):
        super(MM, self).__init__()
        
        self.f1 = paddle.nn.Linear(10, 10)
        self.conv1 = paddle.nn.Conv2D(3, 10, 3, 2)
        self.relu = paddle.nn.ReLU()
        self.seq1 = paddle.nn.Sequential(paddle.nn.Linear(10, 10), paddle.nn.Conv2D(3, 10, 3, 2)) 
        
    def forward(self, ):
        '''
        '''
        pass

MM()

In [12]:

def _get_name(layer:paddle.nn.Layer) -> str:
    ''''''
    return layer.__class__.__name__


def repr_string(layer:paddle.nn.Layer) -> str:
    ''''''
    s = collections.OrderedDict()

    if isinstance(layer, paddle.nn.Conv2D):
        _s = ''
        _s += f'{layer._in_channels}, {layer._out_channels}, '
        _s += f'kernel_size={tuple(layer._kernel_size)}, stride={tuple(layer._stride)}, '
        return '{}({})'.format(_get_name(layer), _s)
    
    elif isinstance(layer, paddle.nn.Linear):
        _s = ''
        _s += f'input_dim={layer.weight.shape[0]}, output_dim={layer.weight.shape[1]}'
        return '{}({})'.format(_get_name(layer), _s)
    
    elif isinstance(layer, paddle.nn.Dropout):
        _s = ''
        _s += f'p={layer.p}'
        return '{}({})'.format(_get_name(layer), _s)
    
    elif isinstance(layer, paddle.nn.ReLU):
        _s = ''
        return '{}({})'.format(_get_name(layer), _s)
    
    elif _get_name(layer) in set(dir(paddle.nn)).difference(set(['Layer', 'Sequential', 'LayerList', 'ParameterList'])):
        _s = ''
        return '{}({})'.format(_get_name(layer), _s)
    
    for _name, _layer in layer.named_children():
        _s = _name
        if isinstance(_layer, (paddle.nn.Sequential, paddle.nn.LayerList, paddle.nn.ParameterList)):
            _s += f' ({_get_name(_layer)})'
            
        s.update({_s: repr_string(_layer)})
    
    # return {f'{_get_name(layer)}': s}
    return s

In [15]:
# TODO

class MM(paddle.nn.Layer):
    '''
    '''
    def __init__(self, ):
        super(MM, self).__init__()
        
        self.f1 = paddle.nn.Linear(10, 10)
        self.conv1 = paddle.nn.Conv2D(3, 10, 3, 2)
        self.relu = paddle.nn.ReLU()
        self.seq1 = paddle.nn.Sequential(paddle.nn.Linear(10, 10), paddle.nn.Conv2D(3, 10, 3, 2)) 
        self.seq2 = paddle.nn.Sequential(self.seq1, paddle.nn.Linear(10, 10), paddle.nn.Conv2D(3, 10, 3, 2))
        self.conv2 = paddle.nn.Conv2D(3, 10, 3, 2)
        self.mlist = paddle.nn.LayerList([paddle.nn.Linear(10, 10) for i in range(3)])
        self.dropout = paddle.nn.Dropout(p=0.5)
        
    def forward(self, ):
        '''
        '''
        pass

    def __repr__(self,):
        ''''''
        # s = repr_string(self)
        return json.dumps(repr_string(self), indent=3)
    
MM()

{
   "f1": "Linear(input_dim=10, output_dim=10)",
   "conv1": "Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2), )",
   "relu": "ReLU()",
   "seq1 (Sequential)": {
      "0": "Linear(input_dim=10, output_dim=10)",
      "1": "Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2), )"
   },
   "seq2 (Sequential)": {
      "0 (Sequential)": {
         "0": "Linear(input_dim=10, output_dim=10)",
         "1": "Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2), )"
      },
      "1": "Linear(input_dim=10, output_dim=10)",
      "2": "Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2), )"
   },
   "conv2": "Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2), )",
   "mlist (LayerList)": {
      "0": "Linear(input_dim=10, output_dim=10)",
      "1": "Linear(input_dim=10, output_dim=10)",
      "2": "Linear(input_dim=10, output_dim=10)"
   },
   "dropout": "Dropout(p=0.5)"
}

In [357]:
mm = MM()

def _get_name(layer:paddle.nn.Layer) -> str:
    ''''''
    return layer.__class__.__name__


def repr_string(layer:paddle.nn.Layer) -> str:
    ''''''
    s = []
    
    if isinstance(layer, paddle.nn.Conv2D):
        _s = ''
        _s += f'{layer._in_channels}, {layer._out_channels}, '
        _s += f'kernel_size={tuple(layer._kernel_size)}, stride={tuple(layer._stride)}'
        return '{}({})'.format(_get_name(layer), _s)
    
    if isinstance(layer, paddle.nn.Linear):
        _s = ''
        _s += f'input_dim={layer.weight.shape[0]}, output_dim={layer.weight.shape[1]}'
        return '{}({})'.format(_get_name(layer), _s)
    
        
    for _name, _layer in layer.named_children():
        if isinstance(_layer, paddle.nn.Sequential):
            _s = '({}):Sequential\n'.format(_name)
            _s += '(\n{}\n)'.format(repr_string(_layer))
            s.append(_s)  
        else:
            _s = '({}): '.format(_name)
            _s += repr_string(_layer)
            s.append(_s)

    return '\n'.join(s)

print(repr_string(mm))

(f1): Linear(input_dim=10, output_dim=10)
(conv1): Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2))
(relu): 
(seq1):Sequential
(
(0): Linear(input_dim=10, output_dim=10)
(1): Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2))
)
(seq2):Sequential
(
(0):Sequential
(
(0): Linear(input_dim=10, output_dim=10)
(1): Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2))
)
(1): Linear(input_dim=10, output_dim=10)
(2): Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2))
)
(conv2): Conv2D(3, 10, kernel_size=(3, 3), stride=(2, 2))
(mlist): (0): Linear(input_dim=10, output_dim=10)
(1): Linear(input_dim=10, output_dim=10)
(2): Linear(input_dim=10, output_dim=10)
(dropout): 


In [16]:
# https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/module.py
# 
    
def _addindent(s_, numSpaces):
    '''
    '''
    s = s_.split('\n')
    # don't do anything for single-line stuff
    if len(s) == 1:
        return s_
    first = s.pop(0)
    s = [(numSpaces * ' ') + line for line in s]
    s = '\n'.join(s)
    s = first + '\n' + s
    return s


# TODO
class Layer(paddle.nn.Layer):
    def extra_repr(self) -> str:
        r"""Set the extra representation of the module
        To print customized extra information, you should re-implement
        this method in your own modules. Both single-line and multi-line
        strings are acceptable.
        """
        return ''
    
    def _get_name(self):
        '''
        '''
        return self.__class__.__name__
    
    def __repr__(self):
        '''
        '''
        # We treat the extra repr like the sub-module, one item per line
        extra_lines = []
        extra_repr = self.extra_repr()
        # empty string will be split into list ['']
        if extra_repr:
            extra_lines = extra_repr.split('\n')
        child_lines = []
        for key, module in self._sub_layers.items():
            mod_str = repr(module)
            mod_str = _addindent(mod_str, 2)
            child_lines.append('(' + key + '): ' + mod_str)
        lines = extra_lines + child_lines

        main_str = self._get_name() + '('
        if lines:
            # simple one-liner info, which most builtin Modules will use
            if len(extra_lines) == 1 and not child_lines:
                main_str += extra_lines[0]
            else:
                main_str += '\n  ' + '\n  '.join(lines) + '\n'

        main_str += ')'
        return main_str


class Conv2D(Layer, paddle.nn.layer.conv.Conv2D):
    def extra_repr(self, ):
        '''
        '''
        s = ('{_in_channels}, {_out_channels}, kernel_size={_kernel_size}'
             ', stride={_stride}')
        if self._padding != (0,) * len(self._padding):
            s += ', padding={_padding}'
        if self._dilation != (1,) * len(self._dilation):
            s += ', dilation={_dilation}'
        if self._groups != 1:
            s += ', groups={_groups}'
        if self._padding_mode != 'zeros':
            s += ', padding_mode={_padding_mode}'
            
        return s.format(**self.__dict__)

    
class Linear(Layer, paddle.nn.layer.Linear):
    def extra_repr(self, ):
        '''
        '''
        _s = ''
        _s += f'input_dim={self.weight.shape[0]}, output_dim={self.weight.shape[1]}'
        return _s
    
    
class ReLU(Layer, paddle.nn.layer.ReLU):
    '''
    '''
    def extra_repr(self, ):
        '''
        '''
        return ''
    
    
class Sequential(Layer, paddle.nn.Sequential):
    '''
    '''
    def _get_item_by_idx(self, iterator, idx):
        """Get the idx-th item of the iterator"""
        size = len(self)
        idx = operator.index(idx)
        if not -size <= idx < size:
            raise IndexError('index {} is out of range'.format(idx))
        idx %= size
        return next(itertools.islice(iterator, idx, None))
    
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            return self.__class__(*list(self._sub_layers.items())[idx])
        else:
            return self._get_item_by_idx(self._sub_layers.values(), idx)
        
    def __setitem__(self, idx: int, layer: paddle.nn.Layer) -> None:
        key = self._get_item_by_idx(self._sub_layers.keys(), idx)
        return setattr(self, key, layer)

    
    
# TODO
class MM(Layer, paddle.nn.Layer):
    '''
    '''
    def __init__(self, ):
        super(MM, self).__init__()
        
        self.f1 = Linear(10, 10)
        self.conv1 = Conv2D(3, 10, 3, 2)
        self.relu = ReLU()
        self.seq1 = Sequential(Linear(10, 10), Conv2D(3, 10, 3, 2)) 
        
    def forward(self, ):
        '''
        '''
        pass

MM()


MM(
  (f1): Linear(input_dim=10, output_dim=10)
  (conv1): Conv2D(3, 10, kernel_size=[3, 3], stride=[2, 2], padding=[0, 0], dilation=[1, 1])
  (relu): ReLU()
  (seq1): Sequential(
    (0): Linear(input_dim=10, output_dim=10)
    (1): Conv2D(3, 10, kernel_size=[3, 3], stride=[2, 2], padding=[0, 0], dilation=[1, 1])
  )
)

In [56]:
issubclass(paddle.nn.Layer, paddle.framework.core.Layer)

True

# Container

In [17]:
# https://github.com/PaddlePaddle/Paddle/blob/develop/python/paddle/fluid/dygraph/container.py
# https://github.com/PaddlePaddle/Paddle/blob/develop/python/paddle/fluid/dygraph/container.py

## Sequential

In [18]:
mm = torch.nn.Sequential(torch.nn.Linear(3, 3), torch.nn.ReLU(), torch.nn.Linear(100, 100))
print(mm)

Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
)


In [19]:
print(mm[-1])

Linear(in_features=100, out_features=100, bias=True)


In [20]:
print(mm[-1:])

Sequential(
  (2): Linear(in_features=100, out_features=100, bias=True)
)


In [21]:
mm[-1] = torch.nn.Linear(100, 100)
print(mm)
print(mm._modules)

Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
)
OrderedDict([('0', Linear(in_features=3, out_features=3, bias=True)), ('1', ReLU()), ('2', Linear(in_features=100, out_features=100, bias=True))])


In [32]:
mm = paddle.nn.Sequential(paddle.nn.Linear(3, 3), paddle.nn.ReLU())
print(mm)
print(mm[-1])

KeyError: '-1'

In [23]:
print(mm[:-1])

KeyError: 'slice(None, -1, None)'

In [93]:
from typing import Union

class Sequential(paddle.nn.Sequential):
    '''
    '''
    def _get_item_by_idx(self, iterator, idx):
        """Get the idx-th item of the iterator"""
        size = len(self)
        idx = operator.index(idx)
        if not -size <= idx < size:
            raise IndexError('index {} is out of range'.format(idx))
        idx %= size
        return next(itertools.islice(iterator, idx, None))
    
    def __getitem__(self, idx: Union[slice, int, str]):
        r'''get 
        mm is sequential instance
        mm[1]
        mm[-1]
        mm[1:]
        mm['L1']
        '''
        if isinstance(idx, str):
            return self._sub_layers[idx]
        elif isinstance(idx, slice):
            return self.__class__(*list(self._sub_layers.items())[idx])
        else:
            return self._get_item_by_idx(self._sub_layers.values(), idx)
        
    def __setitem__(self, idx: Union[int, str], layer: paddle.nn.Layer) -> None:
        r'''set
        mm is sequential instance
        mm[1] = `Layer Instance`
        mm['L1'] = `Layer Instance`
        '''
        if isinstance(idx, str):
            return setattr(self, str(idx), layer)
        else:
            key = self._get_item_by_idx(self._sub_layers.keys(), idx)
            return setattr(self, key, layer)
        
    def __delitem__(self, idx: Union[slice, int, str]) -> None:
        r'''del 
        mm is sequential instance
        del mm[0]
        del mm[-1]
        del mm['2']
        del mm[1:]
        '''
        if isinstance(idx, slice):
            for key in list(self._sub_layers.keys())[idx]:
                delattr(self, key)
        elif isinstance(idx, int):
            key = self._get_item_by_idx(self._sub_layers.keys(), idx)
            delattr(self, key)
        else:
            delattr(self, idx)

In [86]:
mm = Sequential(paddle.nn.Linear(3, 3), 
                  paddle.nn.ReLU(), 
                  paddle.nn.Sequential(paddle.nn.Linear(3, 3), paddle.nn.ReLU()), 
                  paddle.nn.Conv2D(2, 2, 2, 2, 2),
                  paddle.nn.Linear(3, 3))

print(json.dumps(repr_string(mm), indent=3))

{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "Conv2D(2, 2, kernel_size=(2, 2), stride=(2, 2), )",
   "4": "Linear(input_dim=3, output_dim=3)"
}


In [87]:
print(json.dumps(repr_string(mm[-1]), indent=3))

"Linear(input_dim=3, output_dim=3)"


In [88]:
mm[-1] = paddle.nn.ReLU()
print(json.dumps(repr_string(mm), indent=3))

{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "Conv2D(2, 2, kernel_size=(2, 2), stride=(2, 2), )",
   "4": "ReLU()"
}


In [89]:
print(json.dumps(repr_string(mm[:-2]), indent=3))

{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   }
}


In [90]:
print(json.dumps(repr_string(paddle.nn.Sequential(*list(mm.children())[:-1])), indent=3))

{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "Conv2D(2, 2, kernel_size=(2, 2), stride=(2, 2), )"
}


In [91]:
print(json.dumps(repr_string(mm), indent=3))
mm['3'] = paddle.nn.Linear(100, 300)
print(json.dumps(repr_string(mm), indent=3))

{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "Conv2D(2, 2, kernel_size=(2, 2), stride=(2, 2), )",
   "4": "ReLU()"
}
{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "Linear(input_dim=100, output_dim=300)",
   "4": "ReLU()"
}


In [92]:
del mm[-1]
print(json.dumps(repr_string(mm), indent=3))

del mm['2']
print(json.dumps(repr_string(mm), indent=3))

del mm[1:]
print(json.dumps(repr_string(mm), indent=3))

{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "Linear(input_dim=100, output_dim=300)"
}
{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "3": "Linear(input_dim=100, output_dim=300)"
}
{
   "0": "Linear(input_dim=3, output_dim=3)"
}


## ModulList

In [39]:
mm = torch.nn.ModuleList([torch.nn.Linear(3, 3), torch.nn.ReLU()])
print(mm)

ModuleList(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): ReLU()
)


In [40]:
mm[-1]

ReLU()

In [41]:
mm[-1:]

ModuleList(
  (0): ReLU()
)

In [42]:
mm[-1] = torch.nn.Linear(10, 10)
print(mm)

ModuleList(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): Linear(in_features=10, out_features=10, bias=True)
)


In [43]:
mm = paddle.nn.LayerList([paddle.nn.Linear(3, 3), 
                          paddle.nn.ReLU(), 
                          paddle.nn.Sequential(paddle.nn.Linear(3, 3), paddle.nn.ReLU()), 
                          paddle.nn.Linear(3, 3)])
print(mm)

In [44]:
print(mm[-1])

KeyError: '-1'

In [45]:
print(type(mm[:-1]))
print(json.dumps(repr_string(mm[:-1]), indent=3))

<class 'paddle.fluid.dygraph.container.LayerList'>
{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   }
}


In [46]:
mm[-1] = paddle.nn.ReLU()
print(json.dumps(repr_string(mm), indent=3))

{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "Linear(input_dim=3, output_dim=3)",
   "-1": "ReLU()"
}


In [48]:

class LayerList(paddle.nn.LayerList):
    '''
    '''
    
    def _get_abs_string_index(self, idx):
        """Get the absolute index for the list of modules"""
        idx = operator.index(idx)
        if not (-len(self) <= idx < len(self)):
            raise IndexError('index {} is out of range'.format(idx))
        if idx < 0:
            idx += len(self)
        return str(idx)

    
    def __getitem__(self, idx: Union[slice, int]):
        if isinstance(idx, slice):
            return self.__class__(list(self._sub_layers.values())[idx])
        else:
            return self._sub_layers[self._get_abs_string_index(idx)]
        
    def __setitem__(self, idx: int, layer: paddle.nn.Layer) -> None:
        idx = self._get_abs_string_index(idx)
        return setattr(self, str(idx), layer)
    
    T = typing.TypeVar('T')
    def __iadd__(self: T, layers: typing.Iterable[paddle.nn.Layer]) -> T:
        return self.extend(layers)
    
    
    
mm = LayerList([paddle.nn.Linear(3, 3), 
                  paddle.nn.ReLU(), 
                  paddle.nn.Sequential(paddle.nn.Linear(3, 3), paddle.nn.ReLU()), 
                  paddle.nn.Linear(3, 3)])

print(mm)
print(mm[-2])

print(type(mm[:-1]))

print(json.dumps(repr_string(mm[:-1]), indent=3))


mm[-1] = paddle.nn.ReLU()
print(json.dumps(repr_string(mm), indent=3))


mm += [paddle.nn.ReLU()]
print(json.dumps(repr_string(mm), indent=3))


<class '__main__.LayerList'>
{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   }
}
{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "ReLU()"
}
{
   "0": "Linear(input_dim=3, output_dim=3)",
   "1": "ReLU()",
   "2 (Sequential)": {
      "0": "Linear(input_dim=3, output_dim=3)",
      "1": "ReLU()"
   },
   "3": "ReLU()",
   "4": "ReLU()"
}


## ModuleDict

In [ ]:
torch.nn.ModuleDict

# Optimization

In [49]:
# param_group

mm = torch.nn.Sequential(torch.nn.Linear(3, 3), torch.nn.ReLU(), torch.nn.Linear(3, 3))
# optimizer = torch.optim.SGD(params=mm.parameters(), lr=0.1)
optimizer = torch.optim.SGD(params=[
                                    {'params': mm[0].parameters(), 'lr': 0.01}, 
                                    {'params': mm[-1].parameters()}], 
                            lr=0.1, momentum=0.9, weight_decay=0.01)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[2, 5], gamma=0.1)

print(optimizer)
print(scheduler)

SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.01
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0.01

Parameter Group 1
    dampening: 0
    initial_lr: 0.1
    lr: 0.1
    momentum: 0.9
    nesterov: False
    weight_decay: 0.01
)


In [50]:
for e in range(8):
    
    data = torch.rand(size=(10, 3))
    loss = mm(data).sum()
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()
    
    scheduler.step()
    
    print(e, optimizer.param_groups[0]['lr'], optimizer.param_groups[1]['lr'])

0 0.01 0.1
1 0.001 0.010000000000000002
2 0.001 0.010000000000000002
3 0.001 0.010000000000000002
4 0.0001 0.0010000000000000002
5 0.0001 0.0010000000000000002
6 0.0001 0.0010000000000000002
7 0.0001 0.0010000000000000002


In [51]:
mm = paddle.nn.Sequential(paddle.nn.Linear(3, 3), paddle.nn.ReLU(), paddle.nn.Linear(3, 3))

parameters = mm.parameters()

scheduler1 = paddle.optimizer.lr.MultiStepDecay(learning_rate=0.1, milestones=[2, 5], gamma=0.1)
optimizer1 = paddle.optimizer.SGD(learning_rate=scheduler1, 
                                  parameters=list(mm.children())[0].parameters(), 
                                  weight_decay=0.01)

scheduler2 = paddle.optimizer.lr.MultiStepDecay(learning_rate=0.01, milestones=[2, 5], gamma=0.1)
optimizer2 = paddle.optimizer.SGD(learning_rate=scheduler2, 
                                  parameters=parameters[2:], 
                                  weight_decay=0.01)

print(optimizer1)
print(scheduler1)

In [52]:
for e in range(8):
    
    data = paddle.rand(shape=(10, 3))
    loss = mm(data).sum()
    loss.backward()
    
    optimizer1.step()
    optimizer1.clear_grad()
    scheduler1.step()
    
    optimizer2.step()
    optimizer2.clear_grad()
    scheduler2.step()    
    
    print(e, optimizer1.get_lr(), optimizer2.get_lr())

0 0.1 0.01
1 0.010000000000000002 0.001
2 0.010000000000000002 0.001
3 0.010000000000000002 0.001
4 0.0010000000000000002 0.00010000000000000002
5 0.0010000000000000002 0.00010000000000000002
6 0.0010000000000000002 0.00010000000000000002
7 0.0010000000000000002 0.00010000000000000002


# Python-Magic

In [ ]:
# slice

In [ ]:
# __class__

In [ ]:
# multiple inheritance